In [7]:
import os
import pandas as pd
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from tagger.core.mongo.models.order import Order
from pytz import timezone
from datetime import datetime, date
from tagger.models.inventory import Inventory, InventoryStatus
from tagger.models.shipping_notice import ShippingNotice, ShippingNoticeStatus
import xlwt
TZ = timezone("Asia/Seoul")

notice = ShippingNotice.objects.get(id=7)


CJ_COLUMNS = [
    "주문번호", "주문일", "수령자명", "수령자 우편번호", "수령자 연락처", "수령자주소", "상품명", "옵션명", "수량", "배송메모", "박스타입"
]



def get_cj_row(package):
    items = package.shipping_notice_items.all()
    
    item_name = items[0].item.name
    item_size = items[0].item.size
    
    item_count = len(items)
    if item_count > 1:
        item_name += f" 외 {item_count - 1}"
        item_size += f" 외 {item_count - 1}"
        
    return [
        package.code,
        date.today().isoformat().replace("-", "/"),
        package.recipient_name,
        package.postcode,
        f"{package.recipient_mobile[:3]}-{package.recipient_mobile[3:7]}-{package.recipient_mobile[7:11]}",
        package.address,
        item_name,
        item_size,
        1,
        items[0].item.extended_order.order.memo,
        "소"
    ]

def _make_cj(orders):
    data = [get_cj_row(package) for package in notice.packages.all()]
            
    df = pd.DataFrame(data, columns=CJ_COLUMNS).sort_values(by="수령자명")
    df["수령자 우편번호"] = df["수령자 우편번호"].apply(lambda x: x.zfill(5))
    with open(f"../Downloads/cj_upload_{datetime.now().date().isoformat()}.xlsx", "wb") as file:
        df.to_excel(file, index=None)

        
def make_cj(orders):
    data = [get_cj_row(package) for package in notice.packages.all()]
            
    df = pd.DataFrame(data, columns=CJ_COLUMNS).sort_values(by="수령자명")
    df["수령자 우편번호"] = df["수령자 우편번호"].apply(lambda x: x.zfill(5))
    with open(f"../Downloads/cj_upload_{datetime.now().date().isoformat()}.xlsx", "wb") as file:
        df.to_excel(file, index=None)


In [3]:
for p in notice.packages.all():
    print(p.recipient_name)
    for item in p.shipping_notice_items.all():
        print(f"""ITEM {item.item.product_code} / INV {item.inventory.product_code}""")
        print(f"""CROSSCHECK {item.item.extended_order.order.orderstatus} {item.item.extended_order.order.payment.name}""")


신정연
ITEM BEANPOLEL___[빈폴스포츠]투웨이점퍼&조끼다운베스트(블랙)___90 / INV BEANPOLEL___[빈폴스포츠]투웨이점퍼&조끼다운베스트(블랙)___90
CROSSCHECK DELIVERY_PREPARING [빈폴스포츠] 투웨이 점퍼&조끼 다운 베스트(블랙) 
홍은미
ITEM SJSJ___글리터구스점퍼___76(44) / INV SJSJ___글리터구스점퍼___76(44)
CROSSCHECK DELIVERY_PREPARING 글리터 구스 점퍼 
송효은
ITEM BEANPOLEL___오버사이즈소피다운점퍼(브라운)___95 / INV BEANPOLEL___오버사이즈소피다운점퍼(브라운)___95
CROSSCHECK DELIVERY_PREPARING 오버사이즈 소피 다운 점퍼(브라운) 
김혜연
ITEM CCCOLLECT___벨티드플랩코트(카멜)___55 / INV CCCOLLECT___벨티드플랩코트(카멜)___55
CROSSCHECK DELIVERY_PREPARING 벨티드 플랩 코트(카멜) 
안나
ITEM MINE___디테쳐블플리츠후드드레스(SEPIABEIGE)___82(55) / INV MINE___디테쳐블플리츠후드드레스(SEPIABEIGE)___82(55)
CROSSCHECK DELIVERY_PREPARING 디테쳐블 플리츠 후드 드레스(SEPIA BEIGE)
하다정
ITEM BEANPOLEL___[빈폴스포츠]푸퍼미드다운점퍼(네이비)___95 / INV BEANPOLEL___[빈폴스포츠]푸퍼미드다운점퍼(네이비)___95
CROSSCHECK DELIVERY_PREPARING [빈폴스포츠]푸퍼 미드 다운 점퍼(네이비) 
심용식
ITEM BEANPOLEL___[빈폴스포츠](MEN)포켓구스점퍼(아이보리)___100 / INV BEANPOLEL___[빈폴스포츠](MEN)포켓구스점퍼(아이보리)___100
CROSSCHECK DELIVERY_PREPARING [빈폴스포츠] (MEN) 포켓 구스 점퍼(아이보리)  
기민정
ITEM BURBERRY___As

In [15]:
make_cj(notice)
notice.status = ShippingNoticeStatus.SHIPPED
notice.save()

/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField ShippingNotice.shipped_at received a naive datetime (2021-11-17 06:06:47.745142) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


'43D3W'

In [2]:
Inventory.objects.filter(out_order_id="618ccea526685ff0517561b5").first()

<Inventory: Inventory #167 [SHIPPED] 퀄팅 구스다운 점퍼  (211112-023-R)>

In [4]:
data = [get_cj_row(package) for package in notice.packages.all()]

In [5]:
data

[['PKG-211118-4V',
  '2021/11/18',
  '신정연',
  '04085',
  '010-4114-5749',
  '서울 마포구 토정로2길 12 더하우스3차 501호',
  '[빈폴스포츠] 투웨이 점퍼&조끼 다운 베스트(블랙) ',
  '90',
  1,
  '',
  '소'],
 ['PKG-211118-EK',
  '2021/11/18',
  '홍은미',
  '62342',
  '010-3190-3669',
  '광주 광산구 용아로297번길 17 101동1304호',
  '글리터 구스 점퍼 ',
  '76(44) ',
  1,
  '',
  '소'],
 ['PKG-211118-6N',
  '2021/11/18',
  '송효은',
  '07651',
  '010-5500-1835',
  '서울 강서구 강서로 348 108동 1204호',
  '오버사이즈 소피 다운 점퍼(브라운) ',
  '95',
  1,
  '',
  '소'],
 ['PKG-211118-66',
  '2021/11/18',
  '김혜연',
  '30098',
  '010-6641-5346',
  '세종특별자치시 보람로 95 1313동 1501호',
  '벨티드 플랩 코트(카멜) ',
  '55',
  1,
  '배송 전에 미리 연락 바랍니다.',
  '소'],
 ['PKG-211118-E8',
  '2021/11/18',
  '안나',
  '54562',
  '010-7765-1017',
  '전북 익산시 무왕로 1188-8 아름다운수완빌 402호 ',
  '디테쳐블 플리츠 후드 드레스(SEPIA BEIGE)',
  '82(55)',
  1,
  '문옆에 놓아주세요 ',
  '소'],
 ['PKG-211118-4U',
  '2021/11/18',
  '하다정',
  '41528',
  '010-3150-3969',
  '대구 북구 공항로 84 105동 612호',
  '[빈폴스포츠]푸퍼 미드 다운 점퍼(네이비) ',
  '95',
  1,
  '부재 시 집 앞에 놔주세요

In [9]:
wb = xlwt.Workbook()
ws = wb.add_sheet("Sheet1")
for col, val in enumerate(CJ_COLUMNS):
    ws.write(0, col, val)

In [10]:
for row, item in enumerate(data):
    for col, val in enumerate(item):
        ws.write(row+1, col, val)

In [11]:
wb.save("../Downloads/test.xls")